In [1]:
import processing as pP
import pandas as pd
import matplotlib.pyplot as pl
import numpy as np
import confusionMatrix as cM
import decisionTree as tree
import pymongo

In [2]:
#Constants
markers = ['ESAT6', 'CFP10', 'RV1681', 'P24']

In [3]:
#Leer de base de datos de joinedLabApp
MONGO_URL = 'mongodb://idenmon.zapto.org:888/'
client = pymongo.MongoClient(MONGO_URL)
db = client.prodLaboratorio
cursor_img_totals = db.joinedLabApp.find({})
#Crear dataframe a partir de datos lab
df_joinedLabApp = pd.DataFrame(list(cursor_img_totals))
df_joinedLabApp = df_joinedLabApp.drop_duplicates('qr')
df_joinedLabApp.index = df_joinedLabApp['qr']

In [4]:
columns_selected = [m + '_DIAG' for m in markers] + ['Control', 'CODIGO']
columns_selected

['ESAT6_DIAG', 'CFP10_DIAG', 'RV1681_DIAG', 'P24_DIAG', 'Control', 'CODIGO']

In [5]:
df_joinedLabApp = df_joinedLabApp[columns_selected]
df_joinedLabApp.head()

,ESAT6_DIAG,CFP10_DIAG,RV1681_DIAG,P24_DIAG,Control,CODIGO
qr,,,,,,
601170500100160,N,N,N,P,N,101-VP
601170500100147,N,N,N,P,N,105-VP
601170500100103,P,P,P,P,N,107-VP
601170500100157,N,N,N,P,N,108-VP
601170500100173,N,N,N,P,N,109-VP


In [6]:
#Define lista de qrs a usar
qrs = list(df_joinedLabApp.index)

In [7]:
#Proceso completo FIND para lista de qrs
listResults = []

qrsCopy = []
for i,qr in enumerate(qrs):
    result = pP.multiPipeline(str(qr))
    if(not isinstance(result, str)):
        listResults += result
        for r in result:
            qrsCopy.append(qr)
    else:
        print("QR: " + str(qr) + " problem: " + result)
        
df_algoritmos_resultados = pd.DataFrame.from_records(listResults, index = qrsCopy)
df_algoritmos_resultados.head()

,CFP10_area,CFP10_area_color,CFP10_blobs,CFP10_result_area,CFP10_result_blobs,Control_area,Control_area_color,Control_blobs,Control_result_area,Control_result_blobs,...,P24_result_area,P24_result_blobs,RV1681_area,RV1681_area_color,RV1681_blobs,RV1681_result_area,RV1681_result_blobs,blurry,count,tb_result_area
601170500100160,39.080209,33.54,3,N,P,14.447661,40.26,2,N,I,...,P,P,22.344748,35.84,2,N,I,0,0,N
601170500100160,36.779045,34.22,3,N,P,17.138812,38.50,2,N,I,...,P,P,34.610986,38.46,4,N,P,0,1,N
601170500100160,35.245253,38.26,3,N,P,0.000000,37.38,0,N,N,...,P,P,20.779239,47.60,2,N,I,0,2,N
601170500100160,36.426226,36.18,3,N,P,0.000000,41.22,0,N,N,...,P,P,27.386147,42.64,3,N,P,0,3,N
601170500100147,29.635487,34.66,3,N,P,0.000000,39.88,0,N,N,...,P,P,18.408851,35.98,2,N,I,0,0,N


In [8]:
#Join dataframes, resultados esperados vs reales
df_lab_app_alg = df_joinedLabApp.join(df_algoritmos_resultados)
df_lab_app_alg = df_lab_app_alg[df_lab_app_alg.ESAT6_result_area.notnull()]
df_lab_app_alg.head()

,ESAT6_DIAG,CFP10_DIAG,RV1681_DIAG,P24_DIAG,Control,CODIGO,CFP10_area,CFP10_area_color,CFP10_blobs,CFP10_result_area,...,P24_result_area,P24_result_blobs,RV1681_area,RV1681_area_color,RV1681_blobs,RV1681_result_area,RV1681_result_blobs,blurry,count,tb_result_area
601170500100039,N,N,N,N,N,JJEN65M,34.543715,32.06,3.0,N,...,P,P,46.660933,35.70,4.0,P,P,0.0,0.0,P
601170500100039,N,N,N,N,N,JJEN65M,37.297964,38.10,3.0,N,...,P,P,51.024811,41.02,4.0,P,P,0.0,1.0,P
601170500100039,N,N,N,N,N,JJEN65M,37.184777,37.36,3.0,N,...,P,P,47.877563,40.78,4.0,P,P,0.0,2.0,P
601170500100040,N,N,N,N,N,TACV36F,37.569622,32.70,4.0,N,...,P,P,48.457163,39.50,4.0,P,P,0.0,0.0,P
601170500100042,N,N,N,N,N,JBD19F,15.725656,32.84,1.0,N,...,N,I,25.525785,40.94,2.0,N,I,1.0,0.0,N


In [9]:
#Estadísticas globales por marcador por algoritmo. DIAG vs result_area vs result_blobs